<a href="https://colab.research.google.com/github/NicoPozio/WildfireDetectionDL/blob/main/notebook/Classifier_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/NicoPozio/WildfireDetectionDL.git

In [ ]:
import sys

repo_path = '/content/WildfireDetectionDL'
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [ ]:
# Cell: Update Code
import os

REPO_PATH = "/content/WildfireDetectionDL"

print("🔄 Syncing with GitHub...")
!cd {REPO_PATH} && git pull

print("Code updated. You can now run the Sweep or Train cell immediately.")

In [ ]:
#Install Project Dependencies
#hydra-core: Configuration management
#wandb: Experiment tracking
#omegaconf: Dict handling for Hydra
!pip install -q hydra-core wandb omegaconf


from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
    print("Google Drive Mounted")


In [ ]:
import wandb
from google.colab import userdata
import os

try:
    #Fetch key from Colab Secrets
    api_key = userdata.get('WANDB_API_KEY')

    #Set as Environment Variable
    #This ensures Hydra and subprocesses can find it automatically
    os.environ["WANDB_API_KEY"] = api_key

    wandb.login()
    print("Logged in to WandB")

except Exception as e:
    print(f"Authentication Failed: {e}")
    print("Action Required: Go to the 'Secrets' tab (Key icon) on the left.")
    print("Add a new secret named 'WANDB_API_KEY' with your key from https://wandb.ai/authorize")

First we download the dataset, we check if the user has the data in its google drive, if not it's downloaded from a public link

In [ ]:
import os
import gdown
from google.colab import drive
from src.utils import extract_zip



#destination
local_root = "/content/data"
os.makedirs(local_root, exist_ok=True)
!rm -rf {local_root}

#Check if files exist in the mounted drive
#If the users executed the cycle_gan notebook he will have the dataset saved on his drive
drive_dataset_path = "/content/drive/MyDrive/Wildfire_Project/dataset.zip"
drive_synthetic_path = "/content/drive/MyDrive/Wildfire_Project/synthetic_wildfire_2k.zip"

#If not we can download it from this public (non-modifiable) drive folder
public_drive_folder_url = "https://drive.google.com/drive/folders/1bH_5LoY0jJ7NQj1fmhpxpi4T7UvWa9gG?usp=drive_link"


#Check if files exist in the mounted drive
if os.path.isfile(drive_dataset_path) and os.path.isfile(drive_synthetic_path):
    # Assign paths to the Drive locations
    target_dataset_zip = drive_dataset_path
    target_synthetic_zip = drive_synthetic_path

else:

    # Download the entire folder
    # --quiet=False ensures the download progress bar is hidden
    gdown.download_folder(url=public_drive_folder_url, output=local_root, quiet=False)

    # assign paths to the locally downloaded versions
    target_dataset_zip = os.path.join(local_root, "dataset.zip")
    target_synthetic_zip = os.path.join(local_root, "synthetic_wildfire_2k.zip")

    # Validation: Ensure download actually resulted in the required files
    if not (os.path.exists(target_dataset_zip) and os.path.exists(target_synthetic_zip)):
        raise FileNotFoundError(f"Download completed, but expected zip files were not found in {local_root}. Check the public Drive folder contents.")

print(f"\nBeginning extraction")

# Extract Dataset
extract_zip(target_dataset_zip, local_root)

# Extract Synthetic Dataset
extract_zip(target_synthetic_zip, local_root)

print("\nDATA PREPARATION COMPLETE.")
print(f"Extracted data is available in: {local_root}")



In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# Define your data root
DATA_ROOT = "/content/data"

print(f"Scanning {DATA_ROOT} for corrupt images...")

corrupt_count = 0
for root, dirs, files in os.walk(DATA_ROOT):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(root, filename)
            try:
                # Try to fully load the image bytes
                with Image.open(file_path) as img:
                    img.load()
            except OSError:
                print(f"Found corrupt image: {file_path}")
                try:
                    os.remove(file_path)
                    print(f"Deleted {filename}")
                    corrupt_count += 1
                except:
                    print(f"Could not delete {filename}")

print(f"\nScan Complete. Removed {corrupt_count} corrupt files.")

In [ ]:
import subprocess
import os

# 1. Define Paths Constants
REPO_ROOT = "/content/WildfireDetectionDL"
SWEEP_CONFIG_PATH = os.path.join(REPO_ROOT, "conf", "sweep.yaml")

# 2. Validation
if not os.path.exists(SWEEP_CONFIG_PATH):
    print(f"CRITICAL ERROR: Could not find sweep.yaml at {SWEEP_CONFIG_PATH}")
else:
    print(f"--- Registering Sweep from {SWEEP_CONFIG_PATH} ---")

    # 3. Register the sweep
    result = subprocess.run(
        ["wandb", "sweep", SWEEP_CONFIG_PATH],
        capture_output=True,
        text=True,
        # CORRECTION: cwd must be the directory, not the file
        cwd=REPO_ROOT
    )

    output_text = result.stderr + result.stdout
    print("Raw Output:", output_text)

    # 4. Extract Sweep ID
    sweep_id = None
    for line in output_text.split('\n'):
        if "wandb agent" in line:
            parts = line.strip().split("wandb agent ")
            if len(parts) > 1:
                sweep_id = parts[-1].strip()
                break

    # 5. Launch the Agent
    if sweep_id:
        print(f"\nSUCCESS: Detected Sweep ID: {sweep_id}")
        print("Starting Agent... (This will run multiple experiments)")

        # CORRECTION: We use the explicit REPO_ROOT variable defined at the top
        !cd {REPO_ROOT} && wandb agent {sweep_id} --count 20
    else:
        print("\nERROR: Could not find Sweep ID.")

In [ ]:
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# ---------------------------------------------------------
# 1. Configuration
# ---------------------------------------------------------
# Based on your logs: https://wandb.ai/pozioniccolo-sapienza-universit-di-roma/WildfireDetectionDL/sweeps/7hmucehp

# REPLACE THESE WITH YOUR EXACT DETAILS FROM THE LOGS
ENTITY = "pozioniccolo-sapienza-universit-di-roma"
PROJECT = "WildfireDetectionDL"  # <--- Correction: Case Sensitive!
SWEEP_ID = "7hmucehp"            # <--- Correction: Hardcoded from your logs to be safe

# Construct the full path
sweep_path = f"{ENTITY}/{PROJECT}/{SWEEP_ID}"
print(f"Connecting to: {sweep_path}...")

# ---------------------------------------------------------
# 2. Fetch Data
# ---------------------------------------------------------
api = wandb.Api()

try:
    sweep = api.sweep(sweep_path)
    print("✅ Connection Successful! Downloading runs...")

    # Get runs sorted by accuracy
    runs = sorted(sweep.runs, key=lambda run: run.summary.get("val_acc", 0), reverse=True)

    results = []
    for run in runs:
        # Only process runs that finished and have metrics
        if "val_acc" in run.summary:
            cfg = run.config

            # Helper to safely get nested keys
            def get_param(config_dict, key_path):
                keys = key_path.split('.')
                val = config_dict
                for k in keys:
                    if isinstance(val, dict):
                        val = val.get(k)
                    else:
                        return None
                return val

            # Extract data using the safe helper
            results.append({
                "run_id": run.id,
                "model": get_param(cfg, 'model.name'),
                "use_synthetic": get_param(cfg, 'dataset.params.use_synthetic'),
                "lr": get_param(cfg, 'training.learning_rate'),
                "batch_size": get_param(cfg, 'training.batch_size'),
                "optimizer": get_param(cfg, 'training.optimizer'),
                "val_acc": run.summary.get("val_acc")
            })

    df = pd.DataFrame(results)

    # ---------------------------------------------------------
    # 3. Analysis & Visualization
    # ---------------------------------------------------------
    if not df.empty:
        best_run = df.iloc[0]
        print("\n🏆 CHAMPION CONFIGURATION 🏆")
        print(f"Run ID:         {best_run['run_id']}")
        print(f"Model:          {best_run['model']}")
        print(f"Synthetic Data: {best_run['use_synthetic']}")
        print(f"Optimizer:      {best_run['optimizer']}")
        print(f"Learning Rate:  {best_run['lr']}")
        print(f"Val Accuracy:   {best_run['val_acc']:.2f}%")

        # Plot
        plt.figure(figsize=(10, 6))
        sns.barplot(data=df, x="model", y="val_acc", hue="use_synthetic", errorbar=None)
        plt.title("Model Performance: Real vs Synthetic Data")
        plt.ylim(80, 100) # Zoom in to the top range (since your acc is high)
        plt.ylabel("Validation Accuracy (%)")
        plt.xlabel("Model Architecture")
        plt.legend(title="Use Synthetic Data")
        plt.grid(axis='y', alpha=0.3)
        plt.show()

    else:
        print("⚠️ No finished runs found with validation accuracy.")

except Exception as e:
    print(f"❌ Error connecting to WandB: {e}")
    print("Check if Entity, Project, or Sweep ID are correct.")

In [ ]:
import wandb
import os

# 1. Configuration (Must match your logs exactly)
ENTITY = "pozioniccolo-sapienza-universit-di-roma"
PROJECT = "WildfireDetectionDL"

api = wandb.Api()

if 'best_run' in locals() and not df.empty:
    # Get the ID from the pandas row
    run_id = best_run['run_id']

    print(f"🏆 Preparing to test Best Model: {best_run['model']} (Run ID: {run_id})...")

    # 2. Construct the path using known variables
    # Format: entity/project/run_id
    run_path = f"{ENTITY}/{PROJECT}/{run_id}"

    print(f"⬇️ Downloading weights from: {run_path}")

    try:
        # 3. Download 'best_model.pth'
        run_obj = api.run(run_path)
        run_obj.file("best_model.pth").download(replace=True, root=".")

        print("✅ Download Complete. Weights are ready.")

        # 4. Run the Test
        print("🚀 Starting Test Script...")

        # Extract params for the command line
        use_syn = str(best_run['use_synthetic']).lower()
        batch_size = int(best_run['batch_size'])

        !python ./WildfireDetectionDL/test.py \
          model.name={best_run['model']} \
          dataset.params.use_synthetic={use_syn} \
          training.batch_size={batch_size} \
          wandb.mode="online"

    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("Cannot run test: No best run identified in the analysis step.")